In [1]:
from datasets import load_dataset

import nibabel as nib
import scipy.ndimage

from nibabel.processing import resample_to_output
from concurrent.futures import ThreadPoolExecutor
from matplotlib import pyplot as plt
import numpy as np
import os
import glob
import tempfile
from natsort import natsorted
from ipywidgets import interact, IntSlider

from monai.apps import download_and_extract

In [2]:
data_dir = '/home/pawel/Documents/RISA/3D_segmentation'

In [14]:
resource = "https://zenodo.org/records/10069289/files/AeroPath.zip?download=1"
md5 = "3fd5106c175c85d60eaece220f5dfd87"

compressed_file = os.path.join(data_dir, "AeroPath.zip")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, data_dir, md5)

AeroPath.zip: 4.70GB [02:51, 29.4MB/s]                               

2024-06-23 12:55:50,366 - INFO - Downloaded: /home/pawel/Documents/RiSA/3D_segmentation/AeroPath/AeroPath.zip


2024-06-23 12:55:57,824 - INFO - Verified 'AeroPath.zip', md5: 3fd5106c175c85d60eaece220f5dfd87.
2024-06-23 12:55:57,825 - INFO - Writing into directory: /home/pawel/Documents/RiSA/3D_segmentation/AeroPath.


In [3]:
pattern = os.path.join(data_dir, '**/*_CT_HR_label_airways.nii.gz')
train_labels = natsorted(glob.glob(pattern, recursive=True))
print(train_labels)

pattern = os.path.join(data_dir, '**/*_CT_HR.nii.gz')
train_scans = natsorted(glob.glob(pattern, recursive=True))
print(train_scans)

['/home/pawel/Documents/RISA/3D_segmentation/AeroPath/1/1_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/2/2_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/3/3_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/4/4_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/5/5_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/6/6_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/7/7_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/8/8_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/9/9_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/10/10_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroPath/11/11_CT_HR_label_airways.nii.gz', '/home/pawel/Documents/RISA/3D_segmentation/AeroP

In [4]:
for scan_path, label_path in zip(train_scans, train_labels):
    scan = nib.load(scan_path)
    label = nib.load(label_path)

    resampled_scan = resample_to_output(scan, (0.75, 0.75, 0.75), order=1)
    nib.save(resampled_scan, scan_path)

    resampled_label = resample_to_output(label, (0.75, 0.75, 0.75), order=1)
    nib.save(resampled_label, label_path)

KeyboardInterrupt: 

In [49]:
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def save_patched_data(data, affine, base_path, idx, prefix):
    x, y, z = data.shape
    patches = [
        (data[:x//2, :, :z//2], f'{idx}_1_{prefix}.nii.gz'),
        (data[x//2:, :, :z//2], f'{idx}_2_{prefix}.nii.gz'),
        (data[:x//2, :, z//2:], f'{idx}_3_{prefix}.nii.gz'),
        (data[x//2:, :, z//2:], f'{idx}_4_{prefix}.nii.gz')
    ]
    for patch, filename in patches:
        patch[patch > 1024] = 1024
        patch[patch < -1024] = -1024
        patch = (patch - np.min(patch)) / (np.max(patch) - np.min(patch))
        print("min: ", np.min(patch), "max: ", np.max(patch))
        img = nib.Nifti1Image(patch, affine)
        nib.save(img, os.path.join(base_path, filename))

scan_base_path = '../dataset/scan_patched_4'
label_base_path = '../dataset/airways_patched_4'
create_directory(scan_base_path)
create_directory(label_base_path)

for idx, (scan_path, label_path) in enumerate(zip(train_scans, train_labels)):
    scan = nib.load(scan_path)
    data_scan = scan.get_fdata()
    print("Input scan shape: ", data_scan.shape)
    print("min: ", np.min(data_scan), "max: ", np.max(data_scan))
    affine = scan.affine

    label = nib.load(label_path)
    data_label = label.get_fdata()
    print("Input label shape: ", data_label.shape)
    print("min: ", np.min(data_label), "max: ", np.max(data_label))

    save_patched_data(data_scan, affine, scan_base_path, idx, "CT_HR")
    save_patched_data(data_label, affine, label_base_path, idx, "CT_HR")


Input scan shape:  (468, 468, 512)
min:  -1024.0 max:  3071.0
Input label shape:  (468, 468, 512)
min:  0.0 max:  1.0
min:  0.0 max:  1.0
min:  0.0 max:  1.0
min:  0.0 max:  1.0
min:  0.0 max:  1.0
min:  0.0 max:  1.0
min:  0.0 max:  1.0
min:  0.0 max:  1.0
min:  0.0 max:  1.0


In [50]:
def load_and_resample_images(image_paths, new_spacing=(0.75, 0.75, 0.75), order=1):
    resampled_images = []
    for image_path in image_paths:
        img = nib.load(image_path)
        resampled_img = resample_to_output(img, new_spacing, order=order)
        resampled_images.append(resampled_img)
    return resampled_images

def show_slice(image, slice_index):
    plt.figure(figsize=(6, 6))
    plt.imshow(image[:, :, slice_index], cmap='gray')
    plt.axis('off')
    plt.show()

def interactive_viewer(resampled_images):
    image_shape = resampled_images[0].shape
    max_slices = image_shape[2]

    def view_image(image_index, slice_index):
        image = resampled_images[image_index].get_fdata()
        show_slice(image, slice_index)

    interact(view_image, 
             image_index=IntSlider(min=0, max=len(resampled_images)-1, step=1, description='Image'), 
             slice_index=IntSlider(min=0, max=max_slices-1, step=1, description='Slice'))

train_images = ['../dataset/scan_patched_4/0_1_CT_HR.nii.gz', '../dataset/scan_patched_4/0_2_CT_HR.nii.gz',
                '../dataset/scan_patched_4/0_3_CT_HR.nii.gz', '../dataset/scan_patched_4/0_4_CT_HR.nii.gz',
                '../dataset/airways_patched_4/0_1_CT_HR.nii.gz', '../dataset/airways_patched_4/0_2_CT_HR.nii.gz',
                '../dataset/airways_patched_4/0_3_CT_HR.nii.gz', '../dataset/airways_patched_4/0_4_CT_HR.nii.gz']

resampled_images = load_and_resample_images(train_images)

interactive_viewer(resampled_images)

interactive(children=(IntSlider(value=0, description='Image', max=7), IntSlider(value=0, description='Slice', …